In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Функция для создания сессии Spark
def create_spark_session(app_name):
    return SparkSession.builder\
    .appName(app_name)\
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")\
    .getOrCreate()

# Функция для чтения CSV-файлов
def read_csv_file(spark, file_path):
    try:
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Функция для преобразования колонок в числовой тип
def cast_columns_to_double(df, columns):
    for column in columns:
        df = df.withColumn(column, F.col(column).cast("double"))
    return df

# Создание сессии Spark
spark = create_spark_session("BikeAnalysis")

# Чтение CSV-файлов
trips_df = read_csv_file(spark, "/content/trip.csv")
stations_df = read_csv_file(spark, "/content/station.csv")

if trips_df is None or stations_df is None:
    print("Не удалось прочитать один или несколько CSV файлов.  Программа завершается.")
    spark.stop()
    exit()

# Приведение нужных колонок к числовому типу
columns_to_cast = ["lat", "long"]
stations_df = cast_columns_to_double(stations_df, columns_to_cast)

stations_df.show()
trips_df.show()


+---+--------------------+------------------+-------------------+----------+------------+-----------------+
| id|                name|               lat|               long|dock_count|        city|installation_date|
+---+--------------------+------------------+-------------------+----------+------------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|    San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|    San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|    San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|    San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|    San Jose|         8/7/2013|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|        15|    San Jose|         8/7/2013|
|  8| San Salvador at 1st|  

**1. Найти велосипед с максимальным временем пробега.**

In [2]:
from pyspark.sql.functions import unix_timestamp, col, sum as spark_sum
try:
    trips_with_duration = trips_df.withColumn(
        "start_timestamp", unix_timestamp(col("start_date"), "M/d/yyyy H:mm").cast("long")
    ).withColumn(
        "end_timestamp", unix_timestamp(col("end_date"), "M/d/yyyy H:mm").cast("long")
    ).withColumn(
        "duration_minutes", (col("end_timestamp") - col("start_timestamp")) / 60
    )
except Exception as e:
    print(f"Ошибка при преобразовании дат: {e}. Убедитесь, что формат даты в файле trips.csv соответствует 'M/d/yyyy H:mm'.")
    spark.stop()
    exit()

# Группируем по bike_id и находим велосипед с максимальным временем пробега
bike_max = trips_with_duration.groupBy("bike_id").agg(
    spark_sum("duration_minutes").alias("total_minutes")
).orderBy(col("total_minutes").desc()).limit(1)

bike_max.show()


+-------+-------------+
|bike_id|total_minutes|
+-------+-------------+
|    535|     310262.0|
+-------+-------------+



**2. Найти наибольшее геодезическое расстояние между станциями.**

In [3]:
from pyspark.sql.functions import max, radians, sin, cos, sqrt, atan2, col

station_pairs = stations_df.alias("station1").crossJoin(stations_df.alias("station2")).filter(col("station1.id") < col("station2.id"))

# Вычисляем расстояние между станциями
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Радиус Земли в километрах

    dlat = radians(lat2) - radians(lat1)
    dlon = radians(lon2) - radians(lon1)

    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

spark.udf.register("haversine", haversine)

station_pairs = station_pairs.withColumn("distance", haversine(col("station1.lat"), col("station1.long"), col("station2.lat"), col("station2.long")))

# Вычисляем максимальное расстояние между станциями
max_distance = station_pairs.agg(max("distance")).collect()[0][0]

print(f"Максимальное геодезическое расстояние между станциями равно {max_distance:.2f} километрам")

Максимальное геодезическое расстояние между станциями равно 69.92 километрам


**3. Найти путь велосипеда с максимальным временем пробега через станции.**

In [4]:
try:
    trips_with_duration = trips_df.withColumn(
        "start_timestamp", unix_timestamp(col("start_date"), "M/d/yyyy H:mm").cast("long")
    ).withColumn(
        "end_timestamp", unix_timestamp(col("end_date"), "M/d/yyyy H:mm").cast("long")
    ).withColumn(
        "duration_minutes", (col("end_timestamp") - col("start_timestamp")) / 60
    )
except Exception as e:
    print(f"Ошибка при преобразовании дат: {e}. Убедитесь, что формат даты в файле trips.csv соответствует 'M/d/yyyy H:mm'.")
    spark.stop()
    exit()

# Группируем по bike_id и находим велосипед с максимальным временем пробега
bike_max = trips_with_duration.groupBy("bike_id").agg(
    spark_sum("duration_minutes").alias("total_minutes")
).orderBy(col("total_minutes").desc()).limit(1)


# Проверка, что bike_max не пуст
bike_max_list = bike_max.collect()
if not bike_max_list:
    print("Не удалось найти велосипед с максимальным временем пробега. Возможно, данные отсутствуют или некорректны.")
    spark.stop()
    exit()

# Получаем bike_id с максимальным пробегом
try:
    bike_id_max = bike_max_list[0]["bike_id"]
except KeyError:
    print("Столбец 'bike_id' не найден в DataFrame bike_max. Убедитесь, что данные корректны.")
    spark.stop()
    exit()


# Отбираем поездки для этого велосипеда и сортируем по start_date
bike_trips = trips_with_duration.filter(col("bike_id") == bike_id_max) \
    .select("start_date", "start_station_name", "end_date", "end_station_name") \
    .orderBy("start_date")

bike_trips.show(truncate=False)

+---------------+---------------------------------------------+---------------+---------------------------------------------+
|start_date     |start_station_name                           |end_date       |end_station_name                             |
+---------------+---------------------------------------------+---------------+---------------------------------------------+
|1/1/2014 13:42 |Mechanics Plaza (Market at Battery)          |1/1/2014 14:36 |Embarcadero at Sansome                       |
|1/1/2014 18:51 |Embarcadero at Sansome                       |1/1/2014 19:13 |Market at 4th                                |
|1/1/2014 19:48 |Market at 4th                                |1/1/2014 20:01 |South Van Ness at Market                     |
|1/10/2014 20:13|Market at 10th                               |1/10/2014 20:17|Powell Street BART                           |
|1/10/2014 8:09 |Embarcadero at Folsom                        |1/10/2014 8:19 |San Francisco Caltrain (Townsend at 4th

**4. Найти количество велосипедов в системе.**

In [5]:
bike_count = trips_df.groupBy("bike_id").agg(F.count("bike_id").alias("count")).count()

print(f"количество велосипедов в системе: {bike_count}")

количество велосипедов в системе: 700


**5. Найти пользователей потративших на поездки более 3 часов.**

In [6]:
filtered_users = trips_with_duration.where(F.col("zip_code").isNotNull())

# Группируем данные по zip_code и считаем общее время поездок
user_time = filtered_users.groupBy("zip_code").agg(
    F.sum("duration_minutes").alias("total_minutes")
)

# Фильтруем пользователей, потративших более 180 минут (3 часа) на поездки
active_users = user_time.where(F.col("total_minutes") > 180)

active_users.select("zip_code", "total_minutes").show()

+--------+-------------+
|zip_code|total_minutes|
+--------+-------------+
|   94102|     318746.0|
|   95134|      12114.0|
|   84606|       1583.0|
|   80305|       3010.0|
|   60070|        483.0|
|   95519|        505.0|
|   43085|        194.0|
|   91910|        840.0|
|   77339|        230.0|
|   48063|        228.0|
|   85022|        209.0|
|    1090|        340.0|
|    2136|        266.0|
|   11722|        405.0|
|   95138|       2583.0|
|   94610|      60490.0|
|   94404|      59813.0|
|   80301|       2537.0|
|   91326|       1096.0|
|   90742|        183.0|
+--------+-------------+
only showing top 20 rows



In [7]:
# Остановка SparkSession
spark.stop()
